## Binary Search Trees
### A binary search tree relies on the property that keys that are less than the parent are found in the left subtree, and keys that are greater than the parent are found in the right subtree.

* To implement the binary search tree, we will use the nodes and references approach similar to the one we used to implement the linked list, and the expression tree. 
* However, because we must be able create and work with a binary search tree that is empty, our implementation will use two classes. 
* The first class we will call BinarySearchTree, and the second class we will call TreeNode. 
* The BinarySearchTree class has a reference to the TreeNode that is the root of the binary search tree. 

In [ ]:
class TreeNode:

    def __init__(self,key,val,left=None,right=None,parent=None):
        # In a bare bones BST, a key is used to uniquely identify a particular node 
        # in the tree and the value is nothing but data associated with that node.
        # A good analogy: A node is a container. The Value is what you put into that container, 
        # and the Key is how you mark the container so that you can retrieve it later.
        self.key = key
        self.payload = val
        self.leftChild = left
        self.rightChild = right
        self.parent = parent

    def hasLeftChild(self):
        return self.leftChild

    def hasRightChild(self):
        return self.rightChild

    def isLeftChild(self):
        return self.parent and self.parent.leftChild == self

    def isRightChild(self):
        return self.parent and self.parent.rightChild == self

    def isRoot(self):
        return not self.parent

    def isLeaf(self):
        return not (self.rightChild or self.leftChild)

    def hasAnyChildren(self):
        return self.rightChild or self.leftChild

    def hasBothChildren(self):
        return self.rightChild and self.leftChild

    def replaceNodeData(self,key,value,lc,rc):
        self.key = key
        self.payload = value
        self.leftChild = lc
        self.rightChild = rc
        if self.hasLeftChild():
            self.leftChild.parent = self
        if self.hasRightChild():
            self.rightChild.parent = self
    
    def spliceOut(self):
        if self.isLeaf():
            if self.isLeftChild():
                
                self.parent.leftChild = None
            else:
                self.parent.rightChild = None
        elif self.hasAnyChildren():
            if self.hasLeftChild():
                
                if self.isLeftChild():
                    
                    self.parent.leftChild = self.leftChild
                else:
                    
                    self.parent.rightChild = self.leftChild
                    self.leftChild.parent = self.parent
        else:
                    
            if self.isLeftChild():
                        
                self.parent.leftChild = self.rightChild
            else:
                self.parent.rightChild = self.rightChild
                self.rightChild.parent = self.parent

    def findSuccessor(self):
        
        succ = None
        if self.hasRightChild():
            succ = self.rightChild.findMin()
        else:
            if self.parent:
                
                if self.isLeftChild():
                    
                    succ = self.parent
                else:
                    self.parent.rightChild = None
                    succ = self.parent.findSuccessor()
                    self.parent.rightChild = self
        return succ

    def findMin(self):
        
        current = self
        while current.hasLeftChild():
            current = current.leftChild
        return current
    
    # it allows to use a for loop on it 
    # ex: for payload in myTree:
    #       print(payload)
    def __iter__(self):
        
        if self.leftChild:
            yield from self.leftChild

        yield self.payload

        if self.rightChild:
            yield from self.rightChild

## Put Method
### it will check to see if the tree already has a root. If there is not a root then put will create a new TreeNode and install it as the root of the tree.
### If a root node is already in place then put calls the private, recursive, helper function put to search the tree according to the following algorithm…
* Starting at the root of the tree, search the binary tree comparing the new key to the key in the current node.
* If the new key is less than the current node, search the left subtree. If the new key is greater than the current node, search the right subtree.
* When there is no left (or right) child to search, we have found the position in the tree where the new node should be installed.
* To add a node to the tree, create a new TreeNode object and insert the object at the point discovered in the previous step.

### Note: Insertion of a duplicate key is handled by replacing the old value of that key with the new one. The current method ignores this duplicate subkey scenario.

## Get Method
### Once the tree is constructed, the next task is to implement the retrieval of a value for a given key. 
### The get method is even easier than the put method because it simply searches the tree recursively until it gets to a non-matching leaf node or finds a matching key. 
### When a matching key is found, the value stored in the payload of the node is returned.

## Deleting Nodes

### The first task is to find the node to delete by searching the tree. 
### If the tree has more than one node we search using the _get method to find the TreeNode that needs to be removed.
### If the tree only has a single node, that means we are removing the root of the tree, but we still must check to make sure the key of the root matches the key that is to be deleted. 
### In either case if the key is not found the del operator raises an error.
### Once we’ve found the node containing the key we want to delete, there are three cases that we must consider:
* The first case is straightforward. If the current node has no children all we need to do is delete the node and remove the reference to this node in the parent.
* The second case is only slightly more complicated. If a node has only a single child, then we can simply promote the child to take the place of its parent.
* The third case is the most difficult case to handle. If a node has two children, then it is unlikely that we can simply promote one of them to take the node’s place. We can, however, search the tree for a node that can be used to replace the one scheduled for deletion.
    * What we need is a node that will preserve the binary search tree relationships for both of the existing left and right subtrees. 
    * The node that will do this is the node that has the next-largest key in the tree. We call this node the successor.
    * The successor is guaranteed to have no more than one child, so we know how to remove it using the two cases for deletion that we have already implemented. 
    * Once the successor has been removed, we simply put it in the tree in place of the node to be deleted.
    * Notice that we make use of the helper methods findSuccessor and findMin to find the successor. 
    * To remove the successor, we make use of the method spliceOut. 
    * The reason we use spliceOut is that it goes directly to the node we want to splice out and makes the right changes.


In [ ]:
class BinarySearchTree:

    def __init__(self):
        self.root = None
        self.size = 0

    def length(self):
        return self.size

    # it allows us to call the built in len() function with python
    def __len__(self):
        return self.size
        #if self.root:
            #yield from self.root
    
    # built-in method iteration that interates through every node
    # it allows to use the square bracket method, ex: myTree[3], myTree[0]
    def __iter__(self):
        return self.root.__iter__()

    # put method 
    # without _ is for code factoring or refactoring
    def put(self,key,val): 
        if self.root:
            self._put(key,val,self.root)
        else:
            self.root = TreeNode(key,val)
        self.size = self.size + 1

    # with _ is used as a helper function to do the heavy lifting
    def _put(self,key,val,currentNode):
        if key < currentNode.key:
            if currentNode.hasLeftChild():
                   self._put(key,val,currentNode.leftChild)
            else:
                   currentNode.leftChild = TreeNode(key,val,parent=currentNode)
        else:
            if currentNode.hasRightChild():
                   self._put(key,val,currentNode.rightChild)
            else:
                   currentNode.rightChild = TreeNode(key,val,parent=currentNode)

    # used to call the put method
    def __setitem__(self,k,v):
        self.put(k,v)
    
    # get method 
    # without _ is for code factoring or refactoring
    def get(self,key):
        if self.root:
            res = self._get(key,self.root)
            if res:
                
                return res.payload
            else:
                return None
        else:
            return None

    # with _ is used as a helper function to do the heavy lifting
    def _get(self,key,currentNode):
        
        if not currentNode:
            return None
        elif currentNode.key == key:
            return currentNode
        elif key < currentNode.key:
            return self._get(key,currentNode.leftChild)
        else:
            return self._get(key,currentNode.rightChild)

    # used to call the get method
    def __getitem__(self,key):
        return self.get(key)

    # This method will simply call get and return True if get returns a value, 
    # or False if it returns None.
    def __contains__(self,key):
        if self._get(key,self.root):
            return True
        else:
            return False
    
    def delete(self,key):
        
        if self.size > 1:
            
            nodeToRemove = self._get(key,self.root)
            if nodeToRemove:
                self.remove(nodeToRemove)
                self.size = self.size-1
            else:
                raise KeyError('Error, key not in tree')
        elif self.size == 1 and self.root.key == key:
            self.root = None
            self.size = self.size - 1
        else:
            raise KeyError('Error, key not in tree')

    # used to call the delete method
    def __delitem__(self,key):
        self.delete(key)
    
    def remove(self,currentNode):
        
        # first case
        if currentNode.isLeaf(): #leaf
            # check if the current node is a left child or right child
            if currentNode == currentNode.parent.leftChild:
                currentNode.parent.leftChild = None
            else:
                currentNode.parent.rightChild = None
        
        # third case
        elif currentNode.hasBothChildren(): #interior
            
            succ = currentNode.findSuccessor()
            succ.spliceOut()
            currentNode.key = succ.key
            currentNode.payload = succ.payload

        # second case
        else: # this node has one child
            if currentNode.hasLeftChild():

                if currentNode.isLeftChild():
                    currentNode.leftChild.parent = currentNode.parent
                    currentNode.parent.leftChild = currentNode.leftChild

                elif currentNode.isRightChild():
                    currentNode.leftChild.parent = currentNode.parent
                    currentNode.parent.rightChild = currentNode.leftChild

                # the current node is the root, then replace
                else:
                    currentNode.replaceNodeData(currentNode.leftChild.key,
                                    currentNode.leftChild.payload,
                                    currentNode.leftChild.leftChild,
                                    currentNode.leftChild.rightChild)
            else:
                
                if currentNode.isLeftChild():
                    currentNode.rightChild.parent = currentNode.parent
                    currentNode.parent.leftChild = currentNode.rightChild

                elif currentNode.isRightChild():
                    currentNode.rightChild.parent = currentNode.parent
                    currentNode.parent.rightChild = currentNode.rightChild
                
                # the current node is the root, then replace
                else:
                    currentNode.replaceNodeData(currentNode.rightChild.key,
                                    currentNode.rightChild.payload,
                                    currentNode.rightChild.leftChild,
                                    currentNode.rightChild.rightChild)